## Tuning the parameters of our model separately for each dataset in the ETH/UCY datasets

In [3]:
import sys
import os
import dill
import json
import argparse
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv

In [4]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

def process_data(input_data, limit=None):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (8 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    if limit:
        data = data.iloc[:limit]
        data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

In [5]:
def evaluate_our_method(data, params, dataset_title='', single_output = False):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, _ = generative_model.predict(x_data, y_data, params, trajectory_length=12)

                    best_fde = None
                    best_ade = None
                    
                    if single_output:
                        avg_x = np.mean(all_pred_x, axis=0)
                        avg_y = np.mean(all_pred_y, axis=0)
                        best_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                        best_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    else:
                        for i in range(len(all_pred_x)):
                            current_pred_x = all_pred_x[i]
                            current_pred_y = all_pred_y[i]

                            fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_fde == None or fde < best_fde:
                                best_fde = fde

                            ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_ade == None or ade < best_ade:
                                best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

In [6]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results_paper_version/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [7]:
def evaluate_on_dataset(our_method_params, file, trajectron_resultset_name, evaluate_most_likely=False, limit=None):
    base_path = './raw_data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for param_idx, params in enumerate(our_method_params):
        data = pd.read_csv(base_path + file, sep='\t', index_col=False, header=None)
        data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

        data = process_data(data, limit=limit)

        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_name)
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_name, suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        num_predictable_trajectories = get_total_predictable_slices(data)
        #assert len(trajectron_fde) == num_predictable_trajectories
        #assert len(trajectron_ade) == num_predictable_trajectories

        ## Ours
        our_fde_best_of_20, our_ade_best_of_20 = evaluate_our_method(data, params, dataset_title=trajectron_resultset_name)
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
        
        if evaluate_most_likely:
            our_fde_single, our_ade_single = evaluate_our_method(data, params, dataset_title=trajectron_resultset_name, single_output=True)
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_single))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_single))

    return [
        ours_results,
        trajectron_results,
    ]

In [8]:
files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel', 
    'zara1_vel', 
    'zara2_vel'
]

In [9]:
def check_param_results(all_params, all_results):
    print("Best param set idx per dataset (ADE/FDE)")
    for res in all_results:
        ours_res = res[0]
        ours_bo20_fde = ours_res['BEST_OF_20']['FDE']
        ours_bo20_ade = ours_res['BEST_OF_20']['ADE']

        index_min_fde = np.argmin(ours_bo20_fde)
        index_min_ade = np.argmin(ours_bo20_ade)
        print(index_min_fde, "/", index_min_ade)


    print()
    all_param_set_fdes = []
    for param_set_no in range(len(z1_res1[0]['BEST_OF_20']['FDE'])):
        total_fde = 0
        total_ade = 0
        for res in all_results:
            ours_res = res[0]
            ours_bo20_fde = ours_res['BEST_OF_20']['FDE'][param_set_no]
            ours_bo20_ade = ours_res['BEST_OF_20']['ADE'][param_set_no]
            total_fde += ours_bo20_fde
            total_ade += ours_bo20_ade
        all_param_set_fdes.append(total_fde)

    print("Total FDE over all datasets for each param set: ",all_param_set_fdes)
    print()
    lowest_fde_param_set_idx = np.argmin(all_param_set_fdes)
    print("Best param set idx: ", lowest_fde_param_set_idx)
    print("Param set: ", all_params[lowest_fde_param_set_idx])

## Zara 1

In [10]:
z1_params = []

grouping_data = [
    #([0.1, 0.4, 0.65, 0.85, 1.0], [1, 6, 5, 5, 3]), 
    #([0.05, 0.35, 0.7, 0.9, 1.0], [1, 8, 5, 3, 3]), 
    #([0.1, 0.35, 0.7, 0.9, 1.0], [1, 8, 5, 4, 2]), 
    #([0.05, 0.25, 0.5, 0.75, 1.0], [1, 5, 5, 5, 4]),
    #([0.05, 0.30, 0.60, 0.90, 1.0], [1, 6, 5, 5, 3]),
    #([0.05, 0.25, 0.5, 1.0], [1, 7, 7, 5]),
    #([0.05, 0.33, 0.66, 1.0], [1, 7, 7, 5]),
    #([0.1, 0.33, 0.66, 1.0], [1, 8, 6, 5]),
    #([0.05, 0.5, 0.75, 1.0], [1, 9, 6, 4]),
    #([0.05, 0.60, 1.0], [1, 11, 8]),
    ([0.05, 0.60, 1.0], [1, 14, 5]), # done
    #([0.1, 0.70, 1.0], [1, 12, 7]),
    #([0.05, 1.0], [1, 19]),
    #([1.0], [20]),
]

for angle_change_prob in [0.2]: #done, high value
    for angle_change_noise in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        for velocity_change_prob in [0.1]: #done, mid value
            for velocity_change_noise in [0.1]: #done, low value
                for discount_lower_bound in [0.2]: #done, low value
                    for discount_avg_prob in [1.0]: #done, high value
                        for group_data in grouping_data:
                            for noise in [0.05]: #done, mid value
                                for stop_prob in [0.0]: #done, low value
                                    for const_vel_model_prob in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                                        group_p, group_c_count = group_data
                                        one_param_set = {
                                            'NOISE': noise, 
                                            'NO_OF_TRAJECTORIES': 300, 
                                            'CONST_VEL_MODEL_PROB': const_vel_model_prob, 
                                            'STOP_PROB': stop_prob, 
                                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                                            'GROUP_PERCENTAGES': group_p, 
                                            'GROUP_CLUSTER_COUNT': group_c_count
                                        }


                                        z1_params.append(one_param_set)
                        
z1_final_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

In [ ]:
z1_res1 = evaluate_on_dataset(z1_params, 'zara2/test/crowds_zara02.txt', 'zara2_vel_modified', limit=1600, evaluate_most_likely=False)
z1_res2 = evaluate_on_dataset(z1_params, 'univ/test/students001.txt', 'univ_vel', limit=2200, evaluate_most_likely=False)
z1_res3 = evaluate_on_dataset(z1_params, 'hotel/test/biwi_hotel.txt', 'hotel_vel', limit=1600, evaluate_most_likely=False)
z1_res4 = evaluate_on_dataset(z1_params, 'eth/test/biwi_eth.txt', 'eth_vel', limit=None, evaluate_most_likely=False)

Ours - univ_vel:  33%|████████████████████████▋                                                 | 12/36 [00:17<00:41,  1.73s/it]

In [196]:
all_z1_res = [z1_res1, z1_res2, z1_res3, z1_res4]
check_param_results(z1_params, all_z1_res)

Best param set idx per dataset (ADE/FDE)
1 / 1
3 / 3
2 / 2
0 / 0

Total FDE over all datasets for each param set:  [1.7669317442270898, 1.8393056515320971, 1.9338614258390265, 2.0206393648785106, 2.1753698036771283, 2.434702511613644]

Best param set idx:  0
Param set:  {'NOISE': 0.05, 'NO_OF_TRAJECTORIES': 300, 'CONST_VEL_MODEL_PROB': 0.5, 'STOP_PROB': 0.0, 'DISCOUNT_AVG_PROB': 1.0, 'DISCOUNT_LOWER_BOUND': 0.2, 'VELOCITY_CHANGE_PROB': 0.1, 'VELOCITY_CHANGE_NOISE': 0.1, 'ANGLE_CHANGE_PROB': 0.2, 'ANGLE_CHANGE_NOISE': 2, 'GROUP_PERCENTAGES': [0.05, 0.6, 1.0], 'GROUP_CLUSTER_COUNT': [1, 14, 5]}


## Zara2

In [ ]:
z2_params = []

grouping_data = [
    ([0.05, 0.60, 1.0], [1, 14, 5]),
]

for angle_change_prob in [0.1]: #done, mid value
    for angle_change_noise in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        for velocity_change_prob in [0.1]: #done, mid value
            for velocity_change_noise in [0.1]: #done, low value
                for discount_lower_bound in [0.2]: #done, low value
                    for discount_avg_prob in [1.0]: #done, high value
                        for group_data in grouping_data:
                            for noise in [0.1]: #done, mid value
                                for stop_prob in [0.025]: #done, mid value
                                    for const_vel_model_prob in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                                        group_p, group_c_count = group_data
                                        one_param_set = {
                                            'NOISE': noise, 
                                            'NO_OF_TRAJECTORIES': 300, 
                                            'CONST_VEL_MODEL_PROB': const_vel_model_prob, 
                                            'STOP_PROB': stop_prob, 
                                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                                            'GROUP_PERCENTAGES': group_p, 
                                            'GROUP_CLUSTER_COUNT': group_c_count
                                        }


                                        z2_params.append(one_param_set)
                        
z2_final_params = {
    'NOISE': 0.1, 
    'NO_OF_TRAJECTORIES': 200, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.025, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.1, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 14, 5]
}

In [ ]:
z2_res1 = evaluate_on_dataset(z2_params, 'zara1/test/crowds_zara01.txt', 'zara1_vel_modified', limit=1400, evaluate_most_likely=False)
z2_res2 = evaluate_on_dataset(z2_params, 'univ/test/students001.txt', 'univ_vel', limit=2200, evaluate_most_likely=False)
z2_res3 = evaluate_on_dataset(z2_params, 'hotel/test/biwi_hotel.txt', 'hotel_vel', limit=1600, evaluate_most_likely=False)
z2_res4 = evaluate_on_dataset(z2_params, 'eth/test/biwi_eth.txt', 'eth_vel', limit=None, evaluate_most_likely=False)


In [ ]:
all_z2_res = [z2_res1, z2_res2, z2_res3, z2_res4]
check_param_results(z2_params, all_z2_res)

## Univ

In [ ]:
univ_params = []

grouping_data = [
    ([0.05, 0.5, 0.75, 1.0], [1, 9, 6, 4]),
]

for angle_change_prob in [0.2]: #done, high value
    for angle_change_noise in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        for velocity_change_prob in [0.1]: #done, mid value
            for velocity_change_noise in [0.1]: #done, low value
                for discount_lower_bound in [0.1]: #done, low value
                    for discount_avg_prob in [1.0]: #done, high value
                        for group_data in grouping_data:
                            for noise in [0.025]: #done, low value
                                for stop_prob in [0.05]: #done, mid value
                                    for const_vel_model_prob in [0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 1.0]:
                                        group_p, group_c_count = group_data
                                        one_param_set = {
                                            'NOISE': noise, 
                                            'NO_OF_TRAJECTORIES': 300, 
                                            'CONST_VEL_MODEL_PROB': const_vel_model_prob, 
                                            'STOP_PROB': stop_prob, 
                                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                                            'GROUP_PERCENTAGES': group_p, 
                                            'GROUP_CLUSTER_COUNT': group_c_count
                                        }


                                        univ_params.append(one_param_set)
                        
univ_final_params = {
    'NOISE': 0.025, 
    'NO_OF_TRAJECTORIES': 200, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}

In [ ]:
univ_res1 = evaluate_on_dataset(univ_params, 'zara1/test/crowds_zara01.txt', 'zara1_vel_modified', limit=1400, evaluate_most_likely=False)
univ_res2 = evaluate_on_dataset(univ_params, 'zara2/test/crowds_zara02.txt', 'zara2_vel_modified', limit=1600, evaluate_most_likely=False)
univ_res3 = evaluate_on_dataset(univ_params, 'hotel/test/biwi_hotel.txt', 'hotel_vel', limit=1600, evaluate_most_likely=False)
univ_res4 = evaluate_on_dataset(univ_params, 'eth/test/biwi_eth.txt', 'eth_vel', limit=None, evaluate_most_likely=False)


In [ ]:
all_univ_res = [univ_res1, univ_res2, univ_res3, univ_res4]
check_param_results(univ_params, all_univ_res)

## Hotel

In [ ]:
hotel_params = []

grouping_data = [
    ([0.05, 0.60, 1.0], [1, 11, 8]),
]

for angle_change_prob in [0.2]: #done, high value
    for angle_change_noise in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        for velocity_change_prob in [0.1]: #done, mid value
            for velocity_change_noise in [0.1]: #done, low value
                for discount_lower_bound in [0.1]: #done, low value
                    for discount_avg_prob in [1.0]: #done, high value
                        for group_data in grouping_data:
                            for noise in [0.05]: #done, mid value
                                for stop_prob in [0.0]: #done, low value
                                    for const_vel_model_prob in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
                                        const_vel_model_prob
                                        group_p, group_c_count = group_data
                                        one_param_set = {
                                            'NOISE': noise, 
                                            'NO_OF_TRAJECTORIES': 300, 
                                            'CONST_VEL_MODEL_PROB': const_vel_model_prob, 
                                            'STOP_PROB': stop_prob, 
                                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                                            'GROUP_PERCENTAGES': group_p, 
                                            'GROUP_CLUSTER_COUNT': group_c_count
                                        }


                                        hotel_params.append(one_param_set)
                        
hotel_final_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 200, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.0, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.1, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.60, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 11, 8]
}

In [ ]:
hotel_res1 = evaluate_on_dataset(hotel_params, 'zara1/test/crowds_zara01.txt', 'zara1_vel_modified', limit=1400, evaluate_most_likely=False)
hotel_res2 = evaluate_on_dataset(hotel_params, 'zara2/test/crowds_zara02.txt', 'zara2_vel_modified', limit=1600, evaluate_most_likely=False)
hotel_res3 = evaluate_on_dataset(hotel_params, 'univ/test/students001.txt', 'univ_vel', limit=2200, evaluate_most_likely=False)
hotel_res4 = evaluate_on_dataset(hotel_params, 'eth/test/biwi_eth.txt', 'eth_vel', limit=None, evaluate_most_likely=False)


In [ ]:
all_hotel_res = [hotel_res1, hotel_res2, hotel_res3, hotel_res4]
check_param_results(hotel_params, all_hotel_res)

## ETH

In [ ]:
eth_params = []

grouping_data = [
    ([0.05, 0.5, 0.75, 1.0], [1, 9, 6, 4]),
]

for angle_change_prob in [0.2]: #done, high value
    for angle_change_noise in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        for velocity_change_prob in [0.1]: #done, mid value
            for velocity_change_noise in [0.1]: #done, low value
                for discount_lower_bound in [0.15]: #done, mid value
                    for discount_avg_prob in [1.0]: #done, high value
                        for group_data in grouping_data:
                            for noise in [0.05]: #done, mid value
                                for stop_prob in [0.025]: #done, mid value
                                    for const_vel_model_prob in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:

                                        group_p, group_c_count = group_data
                                        one_param_set = {
                                            'NOISE': noise, 
                                            'NO_OF_TRAJECTORIES': 300, 
                                            'CONST_VEL_MODEL_PROB': const_vel_model_prob, 
                                            'STOP_PROB': stop_prob, 
                                            'DISCOUNT_AVG_PROB': discount_avg_prob, 
                                            'DISCOUNT_LOWER_BOUND': discount_lower_bound, 
                                            'VELOCITY_CHANGE_PROB': velocity_change_prob,
                                            'VELOCITY_CHANGE_NOISE': velocity_change_noise, 
                                            'ANGLE_CHANGE_PROB': angle_change_prob, 
                                            'ANGLE_CHANGE_NOISE': angle_change_noise, 
                                            'GROUP_PERCENTAGES': group_p, 
                                            'GROUP_CLUSTER_COUNT': group_c_count
                                        }


                                        eth_params.append(one_param_set)
                        
eth_final_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 200, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.025, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.15, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.05, 0.5, 0.75, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 9, 6, 4]
}

In [ ]:
eth_res1 = evaluate_on_dataset(hotel_params, 'zara1/test/crowds_zara01.txt', 'zara1_vel_modified', limit=1400, evaluate_most_likely=False)
eth_res2 = evaluate_on_dataset(hotel_params, 'zara2/test/crowds_zara02.txt', 'zara2_vel_modified', limit=1600, evaluate_most_likely=False)
eth_res3 = evaluate_on_dataset(hotel_params, 'univ/test/students001.txt', 'univ_vel', limit=2200, evaluate_most_likely=False)
eth_res4 = evaluate_on_dataset(hotel_params, 'hotel/test/biwi_hotel.txt', 'hotel_vel', limit=1600, evaluate_most_likely=False)


In [ ]:
all_eth_res = [eth_res1, eth_res2, eth_res3, eth_res4]
check_param_results(eth_params, all_eth_res)